# Coleta de Dados de Playlists do Spotify

## Descrição Geral:
Este código é um script Python que utiliza a biblioteca Spotipy para acessar e extrair dados de playlists do Spotify. Ele coleta informações sobre as músicas presentes em uma ou mais playlists especificadas pelo usuário, como nome da música, nome do artista, ano de lançamento, gênero e duração da música.

## Dependências Externas:
- `pprint`: Uma biblioteca Python para "pretty-printing" estruturas de dados complexas, usada para imprimir resultados de forma legível.
- `tqdm`: Uma biblioteca Python para exibir barras de progresso durante iterações longas ou loops, útil para monitorar o progresso do processamento.
- `pandas`: Uma poderosa biblioteca Python para manipulação e análise de dados, utilizada para criar e manipular estruturas de dados tabulares.
- `spotipy`: Uma biblioteca Python que serve como um cliente para a API Web do Spotify, permitindo acesso a diversos recursos da plataforma Spotify.
  
## Variáveis de Ambiente:
- `CLIENT_ID`: Identificador único do cliente fornecido pelo Spotify para autenticação.
- `CLIENT_SECRET`: Chave secreta do cliente fornecida pelo Spotify para autenticação.
- `playlist_ids`: Uma lista contendo os IDs das playlists do Spotify que serão processadas.

## Fluxo de Execução:
1. O script importa as bibliotecas necessárias: `pprint`, `tqdm`, `pandas`, `spotipy`, e `SpotifyClientCredentials` da biblioteca `spotipy`.
2. Ele inicializa a instância `SpotifyClientCredentials` com as credenciais do cliente (ID do cliente e chave secreta).
3. Uma instância do cliente do Spotify (`sp`) é criada, utilizando o gerenciador de credenciais do Spotify.
4. O script define um dicionário vazio `tracks_data` para armazenar os dados das músicas.
5. Itera sobre cada `playlist_id` especificado na lista `playlist_ids`.
6. Para cada playlist, ele obtém as músicas presentes na playlist através da função `sp.playlist(playlist_id)`.
7. Extrai os dados relevantes de cada música, como nome, artista, ano de lançamento, gênero e duração, e os armazena no dicionário `tracks_data`.
8. Garante que todas as listas no dicionário `tracks_data` tenham o mesmo comprimento, cortando todas elas para o comprimento mínimo encontrado.
9. Cria um DataFrame do pandas (`df`) a partir dos dados coletados no dicionário `tracks_data`.
10. Organiza as colunas do DataFrame para garantir a ordem desejada.
11. Arredonda a coluna de tempo de música para duas casas decimais.
12. Imprime o DataFrame resultante e sua forma (formato) na saída padrão.

## Saída:
A saída do script é um DataFrame do pandas contendo informações detalhadas sobre as músicas coletadas das playlists do Spotify, incluindo nome do artista, nome da música, gênero, ano de lançamento e duração da música. Além disso, o script imprime a forma (shape) do DataFrame, que mostra o número de linhas e colunas presentes.

## Observações:
- Este script requer autenticação com credenciais válidas do Spotify para acessar as informações das playlists.
- Certifique-se de substituir `CLIENT_ID` e `CLIENT_SECRET` com suas próprias credenciais de cliente do Spotify para garantir o funcionamento correto do script.
- Os IDs das playlists do Spotify devem ser fornecidos na lista `playlist_ids` para que o script possa processá-las.

In [ ]:
from pprint import pprint
from tqdm import tqdm
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
CLIENT_ID = 'CLIENT_ID'
CLIENT_SECRET = 'CLIENT_SCRET'
# REDIRECT_URI = 'seu_redirect_uri'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# IDs das playlists
playlist_ids = ["1Zm5jp4DiMVNTgoIwPsN2f"]

client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

tracks_data = {
    'Nome da Música': [],
    'Nome do Artista': [],
    'Ano de Lançamento': [],
    'Gênero': [],
    'Tempo da Música (minutos)': []
}

for playlist_id in playlist_ids:
    # Obter as músicas da playlist
    playlist = sp.playlist(playlist_id)
    tracks = playlist['tracks']['items']

    # Extrair dados das músicas
    for track in tracks:
        track_info = track['track']
        tracks_data['Nome da Música'].append(track_info['name'])
        tracks_data['Ano de Lançamento'].append(track_info['album']['release_date'][:4])

        # Adiciona o gênero à lista ou "Gênero não disponível" se não houver
        if sp.artist(track_info['artists'][0]['id'])['genres']:
            genero = sp.artist(track_info['artists'][0]['id'])['genres'][0]
        else:
            genero = "Gênero não disponível"
        tracks_data['Gênero'].append(genero)

        tracks_data['Tempo da Música (minutos)'].append(track_info['duration_ms'] / 60000)  # Converter para minutos

        artists = track_info['artists']
        artist_names = ', '.join([artist['name'] for artist in artists])
        tracks_data['Nome do Artista'].append(artist_names)

# Garantir que todas as listas tenham o mesmo comprimento
min_length = min(map(len, tracks_data.values()))
tracks_data = {key: value[:min_length] for key, value in tracks_data.items()}

# Criar DataFrame a partir dos dados coletados
df = pd.DataFrame(tracks_data)

df = df[['Nome do Artista', 'Nome da Música', 'Gênero', 'Ano de Lançamento', 'Tempo da Música (minutos)']]


df['Tempo da Música (minutos)'] = df['Tempo da Música (minutos)'].round(2)
print(df)
print(df.shape)
